In [27]:
import numpy as np
import pandas as pd
import spacy
from nltk import Tree
import itertools
import ast
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))
import re
# %python -m spacy download en  //for installing the english dictionary

In [26]:
%pwd

u'/projects/cps2019_funding/shared/Text_Data/New'

In [28]:
pat = pd.read_csv("/projects/cps2019_funding/shared/Patents_Data/cityAnalysis/cleaned_patent_years/patents2008_full.csv")

In [ ]:
pat.tail()

## Get subject of each sentence per abstract

In [ ]:
pat.shape

In [ ]:
# Get subset of 10%

In [29]:
patSam = pat.sample(frac= .001, random_state=42) 

In [ ]:
patSam.shape

In [30]:
# convert float to str
patSam['patent_abstract'] = patSam[['patent_abstract']].replace(np.nan, '', regex=True)

In [31]:
en_nlp = spacy.load('en')

In [6]:
patSam.head()

,Unnamed: 0,Unnamed: 0.1,IPCs,applications,assignees,cited_patents,cpcs,inventors,nbers,patent_abstract,patent_id,patent_type,uspcs,wipos,assignee_city,field,sector,ipc_section
91332,91332,2207,"[{u'ipc_section': u'H', u'ipc_subclass': u'B',...","[{u'app_type': u'12', u'app_id': u'12/244060'}]","[{u'assignee_key_id': u'271691', u'assignee_ci...",[{u'cited_patent_category': u'cited by other'}...,[{u'cpc_subgroup_title': u'Satellite radio bea...,"[{u'inventor_key_id': u'63202', u'inventor_cit...","[{u'nber_category_title': u'Cmp&Cmm', u'nber_s...",An electronic circuit includes a receiver circ...,8331898,utility,[{u'uspc_mainclass_title': u'Telecommunication...,"[{u'wipo_field_title': u'Telecommunications', ...",Dallas,Electrical engineering,Electrical engineering,[u'H']
19778,19778,9778,"[{u'ipc_section': u'G', u'ipc_subclass': u'Q',...","[{u'app_type': u'12', u'app_id': u'12/080976'}]","[{u'assignee_key_id': u'248593', u'assignee_ci...",[{u'cited_patent_category': u'cited by examine...,[{u'cpc_subgroup_title': u'Finance; Insurance;...,"[{u'inventor_key_id': u'2680096', u'inventor_c...","[{u'nber_category_title': u'Cmp&Cmm', u'nber_s...",Disclosed is a novel life insurance product th...,8180656,utility,[{u'uspc_mainclass_title': u'Data processing: ...,[{u'wipo_field_title': u'IT methods for manage...,Hartford,Electrical engineering,Electrical engineering,[u'G']
5719,5719,5719,"[{u'ipc_section': u'H', u'ipc_subclass': u'G',...","[{u'app_type': u'12', u'app_id': u'12/106984'}]","[{u'assignee_key_id': u'66842', u'assignee_cit...",[{u'cited_patent_category': u'cited by other'}...,[{u'cpc_subgroup_title': u'Coupling parts supp...,"[{u'inventor_key_id': u'1245151', u'inventor_c...","[{u'nber_category_title': u'Elec', u'nber_subc...",A wiring module that can be installed in an el...,7762838,utility,[{u'uspc_mainclass_title': u'Electricity: con...,"[{u'wipo_field_title': u'Electrical machinery,...",Solana Beach,None,None,[u'H']
67276,67276,3340,"[{u'ipc_section': None, u'ipc_subclass': None,...","[{u'app_type': u'29', u'app_id': u'29/308481'}]","[{u'assignee_key_id': u'143968', u'assignee_ci...",[{u'cited_patent_category': u'cited by examine...,"[{u'cpc_subgroup_title': None, u'cpc_category'...","[{u'inventor_key_id': u'3634633', u'inventor_c...","[{u'nber_category_title': None, u'nber_subcate...",,D594674,design,"[{u'uspc_mainclass_title': u'Furnishings', u'u...","[{u'wipo_field_title': None, u'wipo_sector_tit...",New Castle,None,None,[None]
13915,13915,3915,"[{u'ipc_section': u'H', u'ipc_subclass': u'L',...","[{u'app_type': u'12', u'app_id': u'12/037845'}]","[{u'assignee_key_id': u'86418', u'assignee_cit...",[{u'cited_patent_category': u'cited by other'}...,[{u'cpc_subgroup_title': u'Network architectur...,"[{u'inventor_key_id': u'123027', u'inventor_ci...","[{u'nber_category_title': u'Cmp&Cmm', u'nber_s...",A trusted communication system and methods of ...,7996890,utility,[{u'uspc_mainclass_title': u'Information secur...,[{u'wipo_field_title': u'Digital communication...,El Segundo,Electrical engineering,Electrical engineering,[u'H']


# 1. Get noun chunk from each abstract

Notice the example below: if we only obtain subject...the returned words are not descriptive of the idea itself.

## Ex: 1

In [32]:
patSam['patent_abstract'].values[1]

"Disclosed is a novel life insurance product that provides an improved return to a purchaser. The life insurance product contains a benefit payment, and has a fixed benefit amount and a variable benefit amount. A policy protection benefit prevents a purchaser's life insurance product from lapsing, and a minimum death benefit ensures that the purchaser receives the greater of the face value of the product and a predetermined percentage of the account value."

In [33]:
doc = en_nlp(unicode(patSam['patent_abstract'].values[1], "utf-8"))

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

[that, product, benefit, purchaser]


### This example had nothing about insurance or policy or death

### This example has nothing about sensor or vehicle

Now, let us try with noun chunk

In [34]:
for np in doc.noun_chunks:
    print np.text #, np.root.dep_, np.root.head.text

a novel life insurance product
an improved return
a purchaser
The life insurance product
a benefit payment
a fixed benefit amount
a variable benefit amount
A policy protection benefit
a purchaser's life insurance product
the purchaser
the face value
the product
a predetermined percentage
the account value


## Ex 2.

In [ ]:
patSam['patent_abstract'].values[5]

In [ ]:
doc = en_nlp(unicode(patSam['patent_abstract'].values[5], "utf-8"))

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

### This example has nothing about sensor or vehicle

Now, let us try with noun chunk

In [ ]:
for np in doc.noun_chunks:
    print np.text #, np.root.dep_, np.root.head.text

# I will get the words that show up most often from all the noun chunks, and exclude the stopwords


In [35]:
prod = patSam['patent_abstract'].apply(lambda x: list(en_nlp(unicode(x,"utf-8")).noun_chunks))

In [17]:
prod.values[3]

[]

In [ ]:
type(prod.values[0])

In [ ]:
str(prod.values[0][1])

In [ ]:
re.sub("[^\w]", " ",  str(prod.values[0][1])).split()

In [41]:
all_words = []
for i in range(len(prod.values[1])):
    words = re.sub("[^\w]", " ",  str(prod.values[1][i])).split()
   # print words
    all_words = (all_words + words)
    #print(all_words)
    

In [42]:
all_words

['a',
 'novel',
 'life',
 'insurance',
 'product',
 'an',
 'improved',
 'return',
 'a',
 'purchaser',
 'The',
 'life',
 'insurance',
 'product',
 'a',
 'benefit',
 'payment',
 'a',
 'fixed',
 'benefit',
 'amount',
 'a',
 'variable',
 'benefit',
 'amount',
 'A',
 'policy',
 'protection',
 'benefit',
 'a',
 'purchaser',
 's',
 'life',
 'insurance',
 'product',
 'the',
 'purchaser',
 'the',
 'face',
 'value',
 'the',
 'product',
 'a',
 'predetermined',
 'percentage',
 'the',
 'account',
 'value']

In [43]:
word_counter = {}
for word in all_words:
    if word in word_counter:
        word_counter[word] += 1
    else:
        word_counter[word] = 1

popular_words = sorted(word_counter, key = word_counter.get, reverse = True); popular_words

['a',
 'product',
 'benefit',
 'the',
 'purchaser',
 'insurance',
 'life',
 'amount',
 'value',
 'an',
 'policy',
 'percentage',
 'A',
 'return',
 'protection',
 'variable',
 'The',
 'payment',
 'account',
 'novel',
 'improved',
 'face',
 's',
 'predetermined',
 'fixed']

In [ ]:
# Now we need to remove stopwords

In [39]:
stop_words = set(stopwords.words('english')) 
  

In [44]:
filtered_sentence = [w for w in popular_words if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in popular_words: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
   
print(filtered_sentence[:5]) 

['product', 'benefit', 'purchaser', 'insurance', 'life']


In [ ]:
# Build a function that takes a dataset and threshold for how many top words you want.

In [45]:
def keyWord(df, num_words):
    prod = df['patent_abstract'].apply(lambda x: list(en_nlp(unicode(x,"utf-8")).noun_chunks))
    all_words = []
    for i in range(len(prod.values[0])):  
        words = re.sub("[^\w]", " ",  str(prod.values[0][i])).split()
       # print words
        all_words = (all_words + words)
        return all_words
    

In [46]:
keyWord(patSam, 5)

['An', 'electronic', 'circuit']

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

lines = ['The query complexity of estimating weighted averages.',
 'New bounds for the query complexity of an algorithm that learns',
 'DFAs with correction equivalence queries.',
 'general procedure to check conjunctive query containment.']

joint_words = ' '.join(lines)

separated_words = word_tokenize(joint_words)

print(separated_words)

In [ ]:
all_words = []
for i in range(len(prod.values[0])):
    all_words.append(str(prod.values[0][i]).split())
    

In [ ]:
all_words

In [ ]:
prod.values[0][1]

In [ ]:
prod.values[0]

In [ ]:
dict(itertools.zip_longest(*[iter(l)] * 2, fillvalue=""))

In [ ]:
str(prod.values[2][1]).split()

In [ ]:
word_counter = {}
for word in prod.values[0]:
    if word in word_counter:
        word_counter[word] += 1
    else:
        word_counter[word] = 1

#popular_words = 
sorted(word_counter, key = word_counter.get, reverse = True)

In [ ]:
patSam['noun_chunk'] = patSam['patent_abstract'].dropna().apply(lambda x: list(
    en_nlp(unicode(x,"utf-8")).noun_chunks))

In [ ]:
patSam.head()

In [ ]:
test = patSam['noun_chunk'].values[0]

In [ ]:
for i in range(len(test)):
    print(str(test[i]).split())
    

In [ ]:
def iter_nsub(df):
    df['patent_abstract'].dropna().apply(lambda x: list(
    en_nlp(unicode(x,"utf-8")).noun_chunks))

In [ ]:
#output city matched with its index number so that we can use the index number to extract rows from dataframe 
#for different cities
def find_cities(df):
    for i in range(len(df)):
        try:
            yield (i, literal_eval(pat04['assignees'][i])[0]['assignee_city'])
        except SyntaxError:
            yield (i, 'TBD')

In [ ]:
#output a list of tuples with unique cities and corresponding number of patents
def count_cities(l):
    counts = {}
    for tup in l:
        counts[tup[1]] = counts.get(tup[1], 0) + 1
    return sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
cities = list(find_cities(patSam))
cities_counts = count_cities(cities)

In [ ]:
str(cities[0][1])

In [ ]:
cities_counts[:10]

In [ ]:
from spacy.symbols import nsubj, nsubjpass, dobj, iobj, pobj

In [ ]:
doc = en_nlp(u'The cat and the dog sleep in the basket near the door.')

In [ ]:
np_labels = set([nsubj, nsubjpass, dobj, iobj, pobj]) # Probably others too
def iter_nps(doc):
    for word in doc:
        if word.dep_ in np_labels:
            yield word.subtree

In [ ]:
iter_nps(doc)

In [ ]:
patSam['patent_abstract'].values[0]

In [ ]:
unicode(patSam['patent_abstract'].values[1], "utf-8")

In [ ]:
doc = en_nlp(unicode("Other circuits, devices, systems, \
                     methods of operation and processes of manufacture are also disclosed.", "utf-8"))
                     
                     
                     
                     

In [ ]:
sentence

In [ ]:
doc

In [ ]:
doc = en_nlp(unicode(patSam['patent_abstract'].values[1], "utf-8"))

sentence = next(doc.sents) 
for word in sentence:
    print( "%s:%s" % (word,word.dep_))

We see here that simple taking nsub might not result in the most meaningful part of sentence. As shown in insurance example. However...because we are aggregating this might be a trivial issue in the grand scheme.

Lets make a function to store all nsubj for each abstract in dataframe. Then group by...then see what words come to top.

In [ ]:
doc

In [ ]:
patSam['patent_abstract'].apply(lambda x: list(en_nlp(unicode(x,"utf-8"))))

In [ ]:
temp = patSam['patent_abstract'].apply(lambda x: en_nlp(unicode(x,"utf-8")))


In [ ]:
nsub_list = []
for i in range(len(temp)):      
    nsub_list.append([tok for tok in temp.values[i] if (tok.dep_ == "nsubj") ])

    #patSam['nsub'] = [tok for tok in temp.values[i] if (tok.dep_ == "nsubj") ]

In [ ]:
type(nsub_list)

In [ ]:
patSam['nsub'] = pd.DataFrame(np.array(nsub_list))

In [ ]:
patSam.head()

In [ ]:
def nsubMaker(df):
    temp = df['patent_abstract'].apply(lambda x: en_nlp(unicode(x,"utf-8")))
    print(temp.values[0])
    for i in range(len(temp)):      
        print([tok for tok in temp.values[i] if (tok.dep_ == "nsubj") ])
        
        df['nsub'] = [tok for tok in temp.values[i] if (tok.dep_ == "nsubj") ]
    return df
    
    

In [ ]:
patSam['patent_abstract'].apply(lambda x: en_nlp(unicode(x,"utf-8")))

In [ ]:
patSam["nlp"] = patSam['patent_abstract'].apply(lambda x: en_nlp(unicode(x,"utf-8")))

In [ ]:
patSam.head()

In [ ]:
patSam

In [ ]:
for token in doc:
    print(token.text, token.pos_, token.dep_